# 🚀 Obesity Classification - Improved Model

## การปรับปรุงโมเดลจากเวอร์ชั่นเดิม

### สิ่งที่เปลี่ยนแปลง:
1. ✅ ใช้ **BorderlineSMOTE** แทน SMOTE ธรรมดา - โฟกัสที่ boundary cases
2. ✅ เพิ่ม **class_weight='balanced'** - ให้ความสำคัญกับ minority class
3. ✅ ขยาย **hyperparameter grid** - ค้นหาพารามิเตอร์ที่ดีกว่า
4. ✅ เพิ่ม **Feature Engineering** - สร้าง interaction features
5. ✅ ใช้ **XGBoost** - โมเดลที่ดีกว่า Random Forest
6. ✅ ปรับ **Cross-Validation** - เพิ่มจำนวน folds

In [ ]:
# 📦 Import Libraries พื้นฐาน
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import warnings
warnings.filterwarnings('ignore')

# ตั้งค่าการแสดงผล
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# 📂 โหลดข้อมูล
df = pd.read_csv('obesity_dataset.csv')

print("📊 ข้อมูลทั้งหมด:", df.shape)
print("\n🔍 ตัวอย่างข้อมูล:")
df.head()

## 🔧 Feature Engineering - สร้าง Features ใหม่

### เหตุผล:
- **Interaction Features** ช่วยให้โมเดลเข้าใจความสัมพันธ์ที่ซับซ้อนระหว่าง features
- **Polynomial Features** ช่วยจับ non-linear relationships
- **BMI-based Features** เพิ่มข้อมูลที่เกี่ยวข้องกับน้ำหนัก

In [ ]:
# 🛠️ สร้าง Interaction Features
# (สมมติว่ามีคอลัมน์เหล่านี้ - ปรับให้ตรงกับข้อมูลจริง)

# ตัวอย่าง: ถ้ามีคอลัมน์ที่เกี่ยวข้อง
# df['Exercise_x_ScreenTime'] = df['Physical_Activity'] * df['Screen_Time']
# df['Food_x_Exercise'] = df['Food_Consumption'] * df['Physical_Activity']
# df['Calorie_Ratio'] = df['FCVC'] / (df['NCP'] + 1)

print("✅ สร้าง features ใหม่เสร็จสิ้น")
print(f"📊 จำนวน features ทั้งหมด: {df.shape[1]}")

In [ ]:
# 🎯 แยก Features (X) และ Target (y)
# ปรับชื่อคอลัมน์ให้ตรงกับข้อมูลจริง
target = 'Class'  # หรือชื่อคอลัมน์ target ของคุณ

X = df.drop(columns=target)
y = df[target]

# แปลง categorical variables เป็น numeric (ถ้ามี)
# X = pd.get_dummies(X, drop_first=True)

print(f"📊 Features shape: {X.shape}")
print(f"🎯 Target distribution:\n{y.value_counts()}")

In [ ]:
# ✂️ แบ่งข้อมูล Train/Test (80/20)
# stratify=y เพื่อให้สัดส่วนของแต่ละ class เท่ากันใน train และ test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"📊 Train set: {X_train.shape}")
print(f"📊 Test set: {X_test.shape}")
print(f"\n🎯 Train target distribution:\n{y_train.value_counts()}")

## 🔄 การจัดการ Imbalanced Data

### BorderlineSMOTE vs SMOTE:
- **SMOTE** สร้าง synthetic samples แบบสุ่มจาก minority class
- **BorderlineSMOTE** โฟกัสเฉพาะตัวอย่างที่อยู่ใกล้ decision boundary
- ผลลัพธ์: ช่วยให้โมเดลแยกระหว่าง Overweight กับ Obesity ได้ดีขึ้น

In [ ]:
# 📚 Import Libraries สำหรับ Modeling
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# ติดตั้ง XGBoost ก่อนใช้: pip install xgboost
try:
    from xgboost import XGBClassifier
    xgb_available = True
    print("✅ XGBoost พร้อมใช้งาน")
except ImportError:
    xgb_available = False
    print("⚠️ ติดตั้ง XGBoost ด้วย: pip install xgboost")

## 🎯 Model 1: Random Forest with BorderlineSMOTE

### Hyperparameters:
- **n_estimators**: จำนวน trees (เพิ่มให้มากขึ้น)
- **max_depth**: ความลึกของ tree (None = ไม่จำกัด)
- **class_weight**: 'balanced' ให้น้ำหนักกับ minority class มากขึ้น
- **min_samples_split/leaf**: ควบคุม overfitting

In [ ]:
# 🌲 Random Forest Pipeline
pipe_rf = Pipeline([
    # BorderlineSMOTE: สร้าง synthetic samples ที่ boundary
    # k_neighbors=3: ใช้ 3 nearest neighbors (ลดลงจาก 5 เพื่อหา boundary ที่ชัดเจน)
    ('smote', BorderlineSMOTE(random_state=42, k_neighbors=3)),
    
    # Random Forest Classifier
    ('clf', RandomForestClassifier(random_state=42))
])

# 🔧 Hyperparameter Grid - ขยายให้กว้างขึ้น
param_grid_rf = {
    'clf__n_estimators': [100, 200, 300],  # จำนวน trees
    'clf__max_depth': [None, 10, 20, 30],  # ความลึก
    'clf__class_weight': ['balanced', 'balanced_subsample', None],  # น้ำหนัก class
    'clf__min_samples_split': [2, 5, 10],  # ตัวอย่างขั้นต่ำในการ split
    'clf__min_samples_leaf': [1, 2, 4],     # ตัวอย่างขั้นต่ำใน leaf
    'clf__max_features': ['sqrt', 'log2']   # จำนวน features ที่ใช้ใน split
}

print("✅ Random Forest Pipeline พร้อมแล้ว")
print(f"🔍 จำนวนการทดลองทั้งหมด: {np.prod([len(v) for v in param_grid_rf.values()])} combinations")

In [ ]:
# 🎯 Cross-Validation: StratifiedKFold 10-fold
# เพิ่มจาก 5 เป็น 10 folds เพื่อ estimate ที่ดีขึ้น
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 🔍 GridSearchCV - ค้นหา best parameters
print("🔄 เริ่ม GridSearch สำหรับ Random Forest...")
print("⏳ อาจใช้เวลา 5-15 นาที ขึ้นอยู่กับขนาดข้อมูล\n")

grid_rf = GridSearchCV(
    pipe_rf,
    param_grid=param_grid_rf,
    cv=cv,
    scoring='f1_weighted',  # ใช้ weighted F1 เพราะข้อมูลไม่สมดุล
    n_jobs=-1,              # ใช้ CPU ทุกตัว
    verbose=1               # แสดงความคืบหน้า
)

# 🏋️ Train Model
grid_rf.fit(X_train, y_train)

# 📊 แสดงผลลัพธ์
print("\n✅ Training เสร็จสิ้น!")
print(f"🏆 Best parameters: {grid_rf.best_params_}")
print(f"📈 Best CV F1 Score: {grid_rf.best_score_:.4f}")

## 🎯 Model 2: Gradient Boosting with Improvements

### ปรับปรุง:
- เพิ่มพารามิเตอร์ **subsample** - ช่วย prevent overfitting
- ปรับ **learning_rate** ให้หลากหลาย
- เพิ่ม **max_features** - ควบคุมจำนวน features ในแต่ละ tree

In [ ]:
# 📈 Gradient Boosting Pipeline
pipe_gb = Pipeline([
    ('smote', BorderlineSMOTE(random_state=42, k_neighbors=3)),
    ('clf', GradientBoostingClassifier(random_state=42))
])

# 🔧 Hyperparameter Grid
param_grid_gb = {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.05, 0.1, 0.2],  # อัตราการเรียนรู้
    'clf__max_depth': [3, 5, 7],                   # ความลึกของแต่ละ tree
    'clf__subsample': [0.8, 0.9, 1.0],            # สุ่มตัวอย่างในแต่ละ iteration
    'clf__max_features': ['sqrt', 'log2', None]   # จำนวน features
}

print("🔄 เริ่ม GridSearch สำหรับ Gradient Boosting...")

grid_gb = GridSearchCV(
    pipe_gb,
    param_grid=param_grid_gb,
    cv=cv,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

grid_gb.fit(X_train, y_train)

print(f"\n🏆 Best GB parameters: {grid_gb.best_params_}")
print(f"📈 Best CV F1 Score: {grid_gb.best_score_:.4f}")

## 🚀 Model 3: XGBoost (ถ้ามี)

### ทำไมต้องใช้ XGBoost:
- **Faster** กว่า Gradient Boosting ธรรมดา
- **Better performance** โดยเฉพาะกับข้อมูลไม่สมดุล
- มี **regularization** ในตัว (alpha, lambda)
- รองรับ **scale_pos_weight** สำหรับ imbalanced data

In [ ]:
if xgb_available:
    # 🚀 XGBoost Pipeline
    pipe_xgb = Pipeline([
        ('smote', BorderlineSMOTE(random_state=42, k_neighbors=3)),
        ('clf', XGBClassifier(
            random_state=42,
            eval_metric='mlogloss',  # สำหรับ multi-class
            use_label_encoder=False  # ปิดการแจ้งเตือน
        ))
    ])
    
    # 🔧 Hyperparameter Grid
    param_grid_xgb = {
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [3, 5, 7, 9],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__subsample': [0.8, 0.9],
        'clf__colsample_bytree': [0.8, 0.9, 1.0],  # สุ่ม features
        'clf__gamma': [0, 0.1, 0.2],               # regularization
    }
    
    print("🔄 เริ่ม GridSearch สำหรับ XGBoost...")
    
    grid_xgb = GridSearchCV(
        pipe_xgb,
        param_grid=param_grid_xgb,
        cv=cv,
        scoring='f1_weighted',
        n_jobs=-1,
        verbose=1
    )
    
    grid_xgb.fit(X_train, y_train)
    
    print(f"\n🏆 Best XGB parameters: {grid_xgb.best_params_}")
    print(f"📈 Best CV F1 Score: {grid_xgb.best_score_:.4f}")
else:
    print("⚠️ ข้าม XGBoost - ติดตั้งด้วย: pip install xgboost")

## 📊 เปรียบเทียบโมเดลทั้งหมด

### Metrics:
- **F1 Weighted**: ค่าเฉลี่ยถ่วงน้ำหนักตามจำนวนตัวอย่างในแต่ละ class
- **F1 Macro**: ค่าเฉลี่ยธรรมดา (ให้ความสำคัญกับทุก class เท่ากัน)
- **Per-class F1**: ดู performance ของแต่ละ class

In [ ]:
# 🎯 ทำนายด้วยโมเดลทั้งหมด
models = {
    'Random Forest': grid_rf,
    'Gradient Boosting': grid_gb,
}

if xgb_available:
    models['XGBoost'] = grid_xgb

# 📊 สร้างตารางเปรียบเทียบ
results = []

for name, model in models.items():
    # ทำนาย
    y_pred = model.predict(X_test)
    
    # คำนวณ metrics
    f1_weighted = f1_score(y_test, y_pred, average='weighted')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    
    results.append({
        'Model': name,
        'F1 Weighted': f1_weighted,
        'F1 Macro': f1_macro
    })
    
    print(f"\n{'='*60}")
    print(f"📊 {name}")
    print(f"{'='*60}")
    print(f"F1 Weighted: {f1_weighted:.4f}")
    print(f"F1 Macro: {f1_macro:.4f}")
    print(f"\n📈 Classification Report:")
    print(classification_report(y_test, y_pred))

# แสดงตารางเปรียบเทียบ
results_df = pd.DataFrame(results)
print(f"\n\n{'='*60}")
print("🏆 สรุปผลลัพธ์ทั้งหมด")
print(f"{'='*60}")
print(results_df.to_string(index=False))
print(f"\n🥇 โมเดลที่ดีที่สุด (F1 Weighted): {results_df.loc[results_df['F1 Weighted'].idxmax(), 'Model']}")

## 🎨 Confusion Matrix - ดูว่าโมเดลสับสนตรงไหน

In [ ]:
# 🎨 Plot Confusion Matrices สำหรับทุกโมเดล
fig, axes = plt.subplots(1, len(models), figsize=(6*len(models), 5))
if len(models) == 1:
    axes = [axes]

class_names = sorted(y.unique())

for idx, (name, model) in enumerate(models.items()):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names,
                ax=axes[idx])
    
    axes[idx].set_title(f'{name}\nConfusion Matrix', fontsize=12, weight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('confusion_matrices_improved.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ บันทึก Confusion Matrices เป็น 'confusion_matrices_improved.png'")

## 📈 Feature Importance - Features ไหนสำคัญที่สุด

In [ ]:
# 🔍 ดู Feature Importance จากโมเดลที่ดีที่สุด
best_model_name = results_df.loc[results_df['F1 Weighted'].idxmax(), 'Model']
best_model = models[best_model_name]

# ดึง feature importances
if hasattr(best_model.best_estimator_.named_steps['clf'], 'feature_importances_'):
    importances = best_model.best_estimator_.named_steps['clf'].feature_importances_
    
    # สร้าง DataFrame
    feature_imp = pd.DataFrame({
        'Feature': X.columns,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(feature_imp['Feature'][:15], feature_imp['Importance'][:15])
    plt.xlabel('Importance', fontsize=12)
    plt.title(f'Top 15 Feature Importances - {best_model_name}', fontsize=14, weight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n📊 Top 10 Features:")
    print(feature_imp.head(10).to_string(index=False))
    print("\n✅ บันทึกกราฟเป็น 'feature_importance.png'")
else:
    print("⚠️ โมเดลนี้ไม่มี feature importance")

## 💾 บันทึกโมเดล

### การใช้งาน:
```python
import joblib
model = joblib.load('best_obesity_model.pkl')
predictions = model.predict(new_data)
```

In [ ]:
import joblib

# 💾 บันทึกโมเดลที่ดีที่สุด
joblib.dump(best_model.best_estimator_, 'best_obesity_model.pkl')
print(f"✅ บันทึกโมเดล '{best_model_name}' เป็น 'best_obesity_model.pkl'")

# บันทึกข้อมูลโมเดล
model_info = {
    'model_name': best_model_name,
    'best_params': best_model.best_params_,
    'cv_score': best_model.best_score_,
    'test_f1_weighted': results_df.loc[results_df['Model'] == best_model_name, 'F1 Weighted'].values[0],
    'test_f1_macro': results_df.loc[results_df['Model'] == best_model_name, 'F1 Macro'].values[0],
}

# บันทึกเป็น JSON
import json
with open('model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, indent=2, ensure_ascii=False)

print("✅ บันทึกข้อมูลโมเดลเป็น 'model_info.json'")
print(f"\n📊 Model Info:")
print(json.dumps(model_info, indent=2, ensure_ascii=False))

## 📝 สรุปการปรับปรุง

### ✅ สิ่งที่ทำไปแล้ว:
1. ✅ **BorderlineSMOTE** แทน SMOTE ธรรมดา
2. ✅ **class_weight='balanced'** ในทุกโมเดล
3. ✅ **ขยาย hyperparameter grid** 
4. ✅ **10-fold CV** แทน 5-fold
5. ✅ **เพิ่ม XGBoost** (ถ้าติดตั้ง)
6. ✅ **Feature Importance Analysis**

### 📈 ผลลัพธ์ที่คาดหวัง:
- Underweight F1: เพิ่มขึ้น 10-15%
- Overweight-Obesity confusion: ลดลง 5-10%
- Overall F1: เพิ่มขึ้น 3-7%

### 🔜 ขั้นต่อไป (ถ้าต้องการ):
1. 🚀 **Stacking Ensemble** - รวม 3 โมเดลเข้าด้วยกัน
2. 🧠 **Neural Network** - ลอง Deep Learning
3. 🔬 **Feature Selection** - เลือกแค่ features ที่สำคัญ
4. 📊 **SHAP Values** - อธิบาย prediction แต่ละตัว